In [13]:
from sklearn.feature_extraction.text import CountVectorizer

import csv
import pandas as pd
import numpy as np
import sklearn.utils

In [14]:
# Get dataframe and simple pared down version of it
training_dataframe = sklearn.utils.shuffle(pd.read_csv("train-balanced-sarcasm.csv")).iloc[:250]

# Create vectorizer to convert words to vectors
vectorizer = CountVectorizer()
vectorizer.fit(training_dataframe['comment'].values.astype('U'))

# Create training data and labels with dataframe and vectorizer
y = np.matrix(training_dataframe["label"].tolist()).T
x = np.matrix(vectorizer.transform(training_dataframe['comment'].values.astype('U')).toarray())

# Dimensions of input for our model
input_dim = x.shape[1]